In [16]:
data=open('kafka.txt','r').read()
chars=list(set(data))
data_size,vocab_size=len(data),len(chars)
print ("data has "+str(data_size)+" chars, "+str(vocab_size)+" unique ")

data has 137629 chars, 81 unique 


In [18]:
#Dictinary
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print (char_to_ix)
print (ix_to_char)

{'o': 0, 'v': 1, 'g': 2, 'w': 3, 'S': 4, '"': 5, 's': 6, '§': 7, 'r': 8, '5': 9, '3': 10, '9': 11, 't': 12, 'f': 13, 'R': 14, '@': 15, ',': 16, '/': 17, 'm': 18, 'n': 19, '7': 20, '2': 21, 'a': 22, 'q': 23, ';': 24, 'L': 25, 'e': 26, 'U': 27, '4': 28, 'y': 29, 'Ã': 30, '6': 31, 'F': 32, 'u': 33, '.': 34, '\n': 35, 'x': 36, "'": 37, 'j': 38, '?': 39, 'p': 40, 'l': 41, 'M': 42, 'J': 43, 'Y': 44, 'A': 45, '8': 46, 'X': 47, 'i': 48, '0': 49, '!': 50, 'h': 51, 'T': 52, 'N': 53, 'd': 54, 'V': 55, '*': 56, 'O': 57, 'P': 58, 'K': 59, 'G': 60, ':': 61, '1': 62, '-': 63, 'E': 64, 'k': 65, ')': 66, 'H': 67, 'c': 68, 'z': 69, ' ': 70, 'W': 71, 'I': 72, 'C': 73, '(': 74, '$': 75, 'b': 76, 'D': 77, '%': 78, 'B': 79, 'Q': 80}
{0: 'o', 1: 'v', 2: 'g', 3: 'w', 4: 'S', 5: '"', 6: 's', 7: '§', 8: 'r', 9: '5', 10: '3', 11: '9', 12: 't', 13: 'f', 14: 'R', 15: '@', 16: ',', 17: '/', 18: 'm', 19: 'n', 20: '7', 21: '2', 22: 'a', 23: 'q', 24: ';', 25: 'L', 26: 'e', 27: 'U', 28: '4', 29: 'y', 30: 'Ã', 31: '6', 

In [23]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print (vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [24]:
#hyperParameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [25]:
#modelParameters

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #hidden to next hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #hidden to output 
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [42]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]
    

In [34]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'


In [35]:
sample(hprev,char_to_ix['a'],200)

----
 Ws%?*6@QEaB)p"f%-vO:qaCVckjYBx zs( vw9Yhx2wi42v;fqDMvVtL(-KfE)dXp -yOL,v6Pc.m2eO6rtV97?*aD(YfdgPu@Eb7FHig6K6,ywDgBO)S.3IsI0/YiMVMyaWv(mKK*j%Cs6SmTgCTIY-MJUW;q(,r7CNEWBKbGezWnwUip§ed5YT;"2h?DWNDlHB29Ma 
----


In [37]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [57, 19, 26, 70, 18, 0, 8, 19, 48, 19, 2, 16, 70, 3, 51, 26, 19, 70, 60, 8, 26, 2, 0, 8, 70]
targets [19, 26, 70, 18, 0, 8, 19, 48, 19, 2, 16, 70, 3, 51, 26, 19, 70, 60, 8, 26, 2, 0, 8, 70, 4]


In [43]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print ( 'iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter    

iter 0, loss: 109.861241
----
 GCqHC'p!yCH8@PnÃiK4ndp%CdnY!xzt'IPW/eV)@geFroj.Kul@(h*"F5YoQmg3Te3CM9V17nEK3K,@)Uh'*x$ Mgwl0JB?vvHz§ogGsi3ek,%2wc%Px!P?-dbukF'?uQUPXnWE77 pBIc;Q;ewlbl@9jyphc;yQaa(f*t$(st/6V7k6y1y5@1plÃlxBDz"o)Tc1TLhe 
----
iter 1000, loss: 85.269788
----
 is the,gouve pe wnog . Bo gr. dedcoevelitktod ae eo tow borr he, this tDed f ke fives sy uro for ve mire heov routoanl "muctoe. wfonge kin yke aitd , omisk cry h reyir anl hovfive carinn core tre ln y 
----
iter 2000, loss: 68.500019
----
 heth't aile hiifr ted qur. shish. hed witheethe ,uemerherhe shtN the"demhenchepederehe she hidild gisgelhenhastlegesyoutoy cthe y yile nn wiwhe has tesime love hid ere'tetheedememiiHo nelghirheched sn 
----
iter 3000, loss: 60.061205
----
 w ucuy fuskd, licome:ly a foro sudonn Pathe gousteto mond'ur hoes barded bumter; -hipin loswind ofe ons atr biste kusilo troorlatlenry, domlenery athed todly soum woult:eo,k; berl sutianlhale mim of d 
----
iter 4000, loss: 55.889279
----
 hel an he

iter 35000, loss: 43.925263
----
 y sarday the sso-t therely be he had and wien ke sle they dillshy ther's lede tteing vered thes to he whians on way fave rusted oids wamt's meed, of and efly moll to satsunt to moorse soant the mnow t 
----
iter 36000, loss: 42.625828
----
  riselfid cose helrssed was him yourserfade do fime agk as his hak as badse has tivered him to to ollsious, and white; balling, in his dould veadnedm, waschof the wath dider Gut was ain would sithings 
----
iter 37000, loss: 42.571276
----
  His seach humony nowher it sister.e" some. Hey ontor a; shom'en comsalunicry.

Nher mouper hamprshing upakly, as stoln's his theur stery, and surenter at back him imselt lime the rither ever now, eve 
----
iter 38000, loss: 44.140091
----
 fte dod cley on theissrard; or opyGy things and the fricarer the firt, parmatelect this stey,

Gregored and
Guteps waytevered hid paveeded.

Bee fillsither the casssingain ded of ous poplaged desaik G 
----
iter 39000, loss: 45.853747
----
 da

iter 69000, loss: 39.954328
----
 onning, at sham any anget heame appested just of not at won the was soads up of his mother oucanly he was head him to st to the Souslised stotily so her at ut was, in hay Gragh tle of hat he conyy fem 
----
iter 70000, loss: 39.906030
----
  whought jost in shor fiacesway could and with fan ores hole to shed to some forted but the something to kissid to wiomed obreas the blackents him olvout, Gregorge and bockripn sis torest her fall wat 
----
iter 71000, loss: 41.081453
----
 rsy his for sie breining enes upery. Masiet works no it by more ewad wrowerare.  IR PLOut wher had will that no they beyout f by the proilver, ily spepun of ilsoo, the iftece the roon proruping emyepf 
----
iter 72000, loss: 42.735919
----
 inly then had fene hamft the fiatelly, he he, net in the off.E serfadr. I comy but Sal tadded to elfwise he strint no that eleare - heain his negher the could bee claip. He bet- Gregor ho
chatently on 
----
iter 73000, loss: 41.517980
----
 t 